In [1]:
import sys 
sys.path.append('../utils')
from amg_utils import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [3]:
series = pd.read_csv('series_2.csv')

In [11]:
get_tonic_kws_query = """
   
select 
   
keyword,
count(1),
sum(clicks), 
sum(actual_net_revenue),
sum(rpc)
from 
PRODUCTION.BD_S2CINTERNAL.V_CM_TONIC_KEYWORD_DATA a 
left join 

(select 
        *,
        row_number() over (partition by id order by updated desc nulls last) as rn
    from SEM_TOOLS.CM_FB_CAMPAIGN_MANAGEMENT.TONIC_CAMPAIGN
    qualify rn = 1) b 

on a.tonic_campaign_name = b.name

where b.country_code = 'US' 

group by keyword order by 2 desc , 3 desc
"""

all_tonic_kws = eq(get_tonic_kws_query)

In [20]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

def generate_word2vec_model(keywords, vector_size=100, window=5, min_count=1, workers=4):
    """
    Generate a Word2Vec model given a list of keywords.
    
    :param keywords: List of keywords (strings).
    :param vector_size: Dimensionality of the word vectors.
    :param window: Maximum distance between the current and predicted word within a sentence.
    :param min_count: Ignores all words with total frequency lower than this.
    :param workers: Use these many worker threads to train the model.
    :return: Trained Word2Vec model.
    """
    # Tokenize each keyword into words
    tokenized_keywords = [word_tokenize(keyword.lower()) for keyword in keywords]
    
    # Create and train the Word2Vec model
    model = Word2Vec(sentences=tokenized_keywords, vector_size=vector_size, window=window, min_count=min_count, workers=workers)
    
    return model

In [16]:
list_tonic_kws= all_tonic_kws.keyword.tolist()

In [18]:
len(list_tonic_kws)

85415

In [30]:
model = generate_word2vec_model(list_tonic_kws)

[ 0.03187029  0.09166954 -0.00755618 -0.07004929 -0.05864567 -0.06399351
 -0.01381676  0.18968555  0.06686947 -0.00544923  0.0169939   0.02504334
 -0.08187     0.03824371 -0.08866474  0.02281557 -0.10845939 -0.1805891
  0.06981766 -0.11366253 -0.00046    -0.10292171  0.05348151 -0.06746587
 -0.03591822  0.01935931  0.09457166 -0.12307978 -0.12960501 -0.01124608
 -0.0032752  -0.09625012 -0.06042441  0.0660544   0.00133757 -0.0333114
  0.03909612 -0.10821266  0.00811667 -0.14909497  0.05854994 -0.01047515
 -0.00043778  0.16352151  0.28676388  0.07092138 -0.06494767  0.12588486
  0.11937133  0.00474223 -0.03445582 -0.07445528 -0.10039419 -0.0863428
 -0.08616345 -0.04418299  0.07268412 -0.10794359 -0.1287728   0.00569898
 -0.00487446 -0.03064642  0.03330035 -0.18400611  0.02633094 -0.13677357
 -0.04513092  0.06298394  0.01937886  0.02099131 -0.0742927   0.0455761
 -0.06918918 -0.05050999  0.13333695  0.00303999  0.00213567 -0.04421516
 -0.09472311 -0.07701205 -0.11214115  0.00717688  0.030

In [21]:
def combine_embeddings_with_aggregated_data(model, aggregated_data):
    """
    Combine Word2Vec embeddings with aggregated keyword data.
    
    :param model: Pre-trained Word2Vec model.
    :param aggregated_data: DataFrame with aggregated keyword data.
    :return: Dictionary with keywords as keys and combined vectors as values.
    """
    combined_vectors = {}
    
    for index, row in aggregated_data.iterrows():
        keyword = row['keyword']
        tokenized_keyword = word_tokenize(keyword.lower())
        
        # Average the embeddings for the words in the keyword
        keyword_vector = np.mean([model.wv[word] for word in tokenized_keyword if word in model.wv], axis=0)
        
        # Aggregated numerical features for the keyword
        numerical_features = np.array([row['COUNT(1)'], row['SUM(CLICKS)'], row['SUM(ACTUAL_NET_REVENUE)'], row['SUM(RPC)']])
        
        # Combine the keyword vector with its numerical features
        combined_vector = np.concatenate([keyword_vector, numerical_features])
        
        combined_vectors[keyword] = combined_vector
    
    return combined_vectors

In [23]:
import numpy as np

In [27]:
tonic_kws_embeddings_with_aggregated_data_dict =  combine_embeddings_with_aggregated_data(model, all_tonic_kws)

In [32]:
tonic_kws_embeddings_with_aggregated_data_dict['Diet']

array([-1.97623461e-01,  1.59717113e-01,  9.09614339e-02, -6.18010938e-01,
       -3.60698193e-01, -3.25265199e-01, -2.49315366e-01,  4.11389858e-01,
       -5.23198163e-04,  3.43726017e-02,  2.18076035e-02,  1.79773226e-01,
       -7.16705084e-01,  6.37179762e-02, -5.10212719e-01,  6.65458888e-02,
       -4.10711259e-01, -3.30762118e-01,  2.19797418e-01, -4.13896851e-02,
       -1.07884228e-01, -3.16069663e-01,  4.70045507e-01, -4.78902459e-01,
       -9.09307227e-02,  5.59240729e-02,  6.26704514e-01, -4.99188155e-01,
       -3.39453936e-01, -4.40333098e-01,  8.68283287e-02,  7.78460652e-02,
       -3.95224124e-01,  1.77678049e-01,  1.22001506e-01, -4.79342751e-02,
        2.37052217e-01, -3.47422749e-01, -1.00756131e-01, -3.47707540e-01,
        2.83321053e-01, -2.08978012e-01,  2.03229904e-01,  2.59322822e-01,
        1.04314876e+00,  4.80318755e-01, -6.11775704e-02,  2.49242917e-01,
        5.28115034e-01, -1.70129701e-01, -7.43297413e-02, -2.16936976e-01,
       -2.47741546e-02,  

In [ ]:
# separate the vectorized kws with their performance. use the performance 

In [14]:
all_tonic_kws.head(20)

,keyword,COUNT(1),SUM(CLICKS),SUM(ACTUAL_NET_REVENUE),SUM(RPC)
0,Food for Constipation Relief,178868,207169,63259.861861,55704.190619
1,Multiple Myeloma Early Symptoms And Treatments,177366,199929,109696.797190,98323.099156
2,Systane Eye Drops,168809,196472,78260.557262,68210.208331
3,Early Signs Of Ulcerative Colitis,140552,158153,104234.297873,93429.203205
4,Popular Ed Treatments For Senior Men,134373,156881,59689.299444,52572.192600
5,Natural Remedies,127404,150516,30258.072320,26397.547108
6,Best Otc Arthritis Cream,102238,118959,50004.031887,43518.777160
7,Diet for Kidney Disease,97016,116610,35029.955806,28795.382423
8,Most Effective Ed Treatment,88497,106457,37908.072468,32284.222626
9,Cd Rates Highest Nearby Banks,83485,94369,40142.753280,35871.572714


In [9]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np

# Given keywords (without considering their counts)
keywords = [
    "Tires",
    "Discount Tires",
    "Truck Tires",
    "Bridgestone Tires",
    "Car Tires",
    "Tires Direct Discount",
    "Car Tire",
    "Discounted Tires",
    "Discount On Tires",
    "Cheap Discount Tires Online",
    "Tires Packages",
    "Super Cheap Tires Near Me",
    "Discount Car Tires"
]

# Tokenization of each keyword
tokenized_keywords = [word_tokenize(keyword.lower()) for keyword in keywords]

# Create and train the model
model = Word2Vec(sentences=tokenized_keywords, vector_size=100, window=5, min_count=1, workers=4)

# Example of using the model for new campaigns with new keywords
# Assuming a new keyword "Budget Tires"
new_keyword = "Budget Tires"
tokenized_new_keyword = word_tokenize(new_keyword.lower())

# Initialize an empty vector for the new keyword
new_keyword_vector = np.zeros(model.vector_size)

# Sum the vectors of the words in the new keyword
for word in tokenized_new_keyword:
    if word in model.wv:
        new_keyword_vector += model.wv[word]

# Average the vector for the new keyword
new_keyword_vector /= len(tokenized_new_keyword)

# The new_keyword_vector can now be used as a feature for the machine learning model

new_keyword_vector

array([-2.68113625e-04,  1.18215678e-04,  2.55167484e-03,  4.50463640e-03,
       -4.65147477e-03, -3.55840451e-03,  3.22943623e-03,  4.48649423e-03,
       -2.50771409e-03, -1.88168581e-03,  3.69025231e-03, -7.66735699e-04,
       -2.26830668e-03,  3.27702588e-03, -2.43008020e-03, -9.08008835e-04,
        1.43828988e-03,  4.95936896e-04, -4.14260756e-03, -4.72440897e-03,
        3.65588302e-03,  2.53513106e-03,  3.37884668e-03,  3.81432765e-04,
        3.17544513e-03, -1.70268293e-03, -4.73200693e-04,  2.88428669e-03,
       -3.76081886e-03, -1.96805177e-03, -3.75579111e-03, -4.65021119e-04,
        4.76905936e-03, -3.65958340e-03, -1.16688432e-03, -9.68870532e-04,
        4.03871853e-03, -2.96544796e-03,  2.25812200e-05, -2.37686699e-03,
       -4.80177533e-03,  2.50364654e-03, -4.37979260e-03, -2.19591265e-03,
       -1.75499918e-05, -1.48090723e-04, -3.83062009e-03,  4.80737165e-03,
        2.49102898e-03,  4.61657159e-03, -4.07895865e-03,  2.24789907e-03,
       -2.06853799e-03,  

In [33]:
historical_embeddings = {row['keyword']: get_keyword_embedding(model, row['keyword'])
                         for index, row in all_tonic_kws.iterrows()}

NameError: name 'get_keyword_embedding' is not defined

In [34]:
check_table(name='PRODUCTION.BD_S2CINTERNAL.V_CM_TONIC_KEYWORD_DATA')

,,,,,,,
0,eventdate,object,2023-01-09,2023-01-09,2023-01-09,2023-01-09,2023-01-09
1,fb_account,object,None,None,None,None,None
2,fb_campaign_name,object,ed5_d2l_056_cg01_ac01,ed_d2l_056_cg01_ac01,arthritis_d2l_056_cg01_ac01,breast_cancer_d2l_074_cg01_ac02-ca-ai,crohns_056_cg01_ac01
3,fb_adset_name,object,23852723235440182,23852236912890182,23852117973330182,23852425526610375,23852106590990182
4,tonic_campaign_name,object,fb-us-health-ed-5,fb-us-health-erectile_dysfunction,fb-us-health-arthritis_treatment,fb-ca-health-breast_cancer,fb-us-health-crohns_treatment
5,media_buyer_name,object,None,None,None,Arthur Illasco,None
6,keyword,object,Pills for Sex,Most Effective Ed Treatment,Best Otc Arthritis Cream,Early Breast Cancer Signs and Symptoms,Crohns Symptoms
7,clicks,int64,1,1,1,1,1
8,actual_net_revenue,float64,0.09,0.23,0.57,0.02,0.75
9,rpc,float64,0.09,0.23,0.57,0.02,0.75


In [ ]:
# This code snippet demonstrates how to generate a vector representation for a new keyword ("Budget Tires") using a pre-trained Word2Vec model. The process involves tokenizing the new keyword, summing the Word2Vec embeddings for each token (word) in the keyword, and then averaging these vectors to get a single vector that represents the entire keyword. This final vector can then be used as a feature in machine learning models. 